<img src="header.png" align="left"/>

# Anwendungsbeispiel Import of text data with sentiment classification

In diesem Beispiel werden wir Textdaten behandeln und versuchen die Stimmung eines kurzen Stückes Text zu bestimmen. Damit können zum Beispiel eMails oder Social Media Beiträge gefiltert werden. Diese Variane wurde mit Hilfe eines Transformers implementiert.





- [2] [https://stackabuse.com/python-for-nlp-movie-sentiment-analysis-using-deep-learning-in-keras/](https://stackabuse.com/python-for-nlp-movie-sentiment-analysis-using-deep-learning-in-keras/)
- [3] https://gdcoder.com/sentiment-clas/
- [4] [https://nlp.stanford.edu/pubs/glove.pdf](https://nlp.stanford.edu/pubs/glove.pdf)


Zitierungen GloVe [1] und Datensatz[2]:
```
[1] Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation.

[2] Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher, Learning Word Vectors for Sentiment Analysis, Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies, June 2011, Portland, Oregon, USA, Association for Computational Linguistics, http://www.aclweb.org/anthology/P11-1015

```









# Import der Module

In [1]:
!pip install bert-for-tf2

In [56]:
#
# Import der Module
#
import os
import logging
import re
import string
from urllib.request import urlretrieve
import tarfile
import zipfile
from glob import glob

import pandas as pd
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report




#
# Abdrehen von Fehlermeldungen
#
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=Warning)
simplefilter(action='ignore', category=RuntimeWarning)


#
# Tensorflow und Keras
#
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, SpatialDropout1D
from tensorflow.keras.layers import Flatten, LSTM
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.models import load_model

import bert




#
# Für GPU Support
#
tflogger = tf.get_logger()
tflogger.setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR )
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)


#
# Einstellen der Grösse von Diagrammen
#
plt.rcParams['figure.figsize'] = [16, 9]


#
# Ausgabe der Versionen
#
print('working on keras version {} on tensorflow {} using hub {}'.format ( tf.keras.__version__, tf.version.VERSION, hub.__version__ ) )

working on keras version 2.11.0 on tensorflow 2.11.0 using hub 0.12.0


# Konstanten

In [32]:
#
# Konstanten für Dateien
#
urlDataSource = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
localExtractionFolder = 'data/moviereviews'
localDataArchive = localExtractionFolder + '/aclImdb_v1.tar.gz'
textData = localExtractionFolder + '/aclImdb/'
size = 2000

# Hilfsfunktionen

In [4]:
#
# Laden der Daten von einer URL
#
def download_dataset(url,dataset_file_path,extraction_directory):
    if (not os.path.exists(extraction_directory)):
        os.makedirs(extraction_directory)
    if os.path.exists(dataset_file_path):
        print("archive already downloaded.")
    else:
        print("started loading archive from url {}".format(url))
        filename, headers = urlretrieve(url, dataset_file_path)
        print("finished loading archive from url {} to {}".format(url,filename))

def extract_dataset(dataset_file_path, extraction_directory):
    if (not os.path.exists(extraction_directory)):
        os.makedirs(extraction_directory)
    if (dataset_file_path.endswith("tar.gz") or dataset_file_path.endswith(".tgz")):
        tar = tarfile.open(dataset_file_path, "r:gz")
        tar.extractall(path=extraction_directory)
        tar.close()
    elif (dataset_file_path.endswith("tar")):
        tar = tarfile.open(dataset_file_path, "r:")
        tar.extractall(path=extraction_directory)
        tar.close()
    print("extraction of dataset from {} to {} done.".format(dataset_file_path,extraction_directory) )


# Laden und erster Check

In [5]:
#
# Laden der Daten ausführen
#
download_dataset(urlDataSource,localDataArchive,localExtractionFolder)

archive already downloaded.


In [6]:
#
# Extrahieren der Daten
#
extract_dataset(localDataArchive,localExtractionFolder)

extraction of dataset from data/moviereviews/aclImdb_v1.tar.gz to data/moviereviews done.


# Wie sehen die Daten auf dem Filesystem aus?

<img src="info.png" align="left"/> 

In [33]:
#
# Sammeln der Daten aus den Files
#
def load_texts_labels_from_folders(path, folders):
    print('scanning path {}'.format(path))
    texts,labels = [],[]
    for idx,label in enumerate(folders):
        print('scanning {}'.format(idx))
        count = 0
        for fname in glob(os.path.join(path, label, '*.*')):
            if count > size:
                break
            texts.append(open(fname, 'r').read())
            labels.append(idx)
            count += 1
    return texts, np.array(labels).astype(np.int8)

In [34]:
#
# Laden der positiven und negativen Beispiele
#
classes = ['neg','pos']
x_train,y_train = load_texts_labels_from_folders( textData + 'train', classes)
x_test,y_test = load_texts_labels_from_folders( textData + 'test', classes)

scanning path data/moviereviews/aclImdb/train
scanning 0
scanning 1
scanning path data/moviereviews/aclImdb/test
scanning 0
scanning 1


In [35]:
len(x_train),len(y_train),len(x_test),len(y_test)

(4002, 4002, 4002, 4002)

In [36]:
#
# Prüfen des Datentypen
#
(type(x_train),type(y_train))

(list, numpy.ndarray)

In [37]:
#
# Prüfen der Klassen
#
np.unique(y_train)

array([0, 1], dtype=int8)

In [38]:
#
# negative Beispiele
#
for index in range (0,1):
    print(x_train[index])
    print("label {}".format(y_train[index]))
    print()

Working with one of the best Shakespeare sources, this film manages to be creditable to it's source, whilst still appealing to a wider audience.<br /><br />Branagh steals the film from under Fishburne's nose, and there's a talented cast on good form.
label 0



In [13]:
#
# positive Beispiele
#
for index in range (13001,13002):
    print(x_train[index])
    print("label {}".format(y_train[index]))
    print()


What we have here is a damn good little nineties thriller that, while perhaps lacking in substance, still provides great entertainment throughout it's running time and overall does everything you could possibly want a film of this nature to do. I saw this film principally because it was directed by John Dahl - a highly underrated director behind great thrillers such as The Last Seduction, Rounders and Roadkill. I figured that if this film was up to standard of what I've already seen from the director, it would be well worth watching - and Red Rock West is certainly a film that Dahl can be proud of. The plot focuses on the overly moral Michael; a man travelling across America looking for work. He ends up finding it one day when he stumbles upon a bar in Red Rock County - only catch is that the job is to murder a man's wife. He's been mistaken for a killer named Lyle, but instead of doing the job; he plays both sides against each other and eventually plans to make a getaway. However, his

In [39]:
# Functions for constructing BERT Embeddings: input_ids, input_masks, input_segments and Inputs
MAX_SEQ_LEN=500 # max sequence length

def get_masks(tokens):
    """Masks: 1 for real tokens and 0 for paddings"""
    return [1]*len(tokens) + [0] * (MAX_SEQ_LEN - len(tokens))
 
def get_segments(tokens):
    """Segments: 0 for the first sequence, 1 for the second"""  
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (MAX_SEQ_LEN - len(tokens))

def get_ids(tokens, tokenizer):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (MAX_SEQ_LEN - len(token_ids))
    return input_ids

def create_single_input(sentence, tokenizer, max_len):
    """Create an input from a sentence"""
    stokens = tokenizer.tokenize(sentence)
    stokens = stokens[:max_len]
    stokens = ["[CLS]"] + stokens + ["[SEP]"]
 
    ids = get_ids(stokens, tokenizer)
    masks = get_masks(stokens)
    segments = get_segments(stokens)

    return ids, masks, segments
 
def convert_sentences_to_features(sentences, tokenizer):
    """Convert sentences to features: input_ids, input_masks and input_segments"""
    input_ids, input_masks, input_segments = [], [], []
 
    for sentence in sentences:
      ids,masks,segments=create_single_input(sentence,tokenizer,MAX_SEQ_LEN-2)
      assert len(ids) == MAX_SEQ_LEN
      assert len(masks) == MAX_SEQ_LEN
      assert len(segments) == MAX_SEQ_LEN
      input_ids.append(ids)
      input_masks.append(masks)
      input_segments.append(segments)

    return [np.asarray(input_ids, dtype=np.int32), 
          np.asarray(input_masks, dtype=np.int32), 
          np.asarray(input_segments, dtype=np.int32)]

In [40]:
def create_tonkenizer(bert_layer):
    """Instantiate Tokenizer with vocab"""
    vocab_file=bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case=bert_layer.resolved_object.do_lower_case.numpy() 
    tokenizer=bert.bert_tokenization.FullTokenizer(vocab_file,do_lower_case)
    return tokenizer

In [41]:
def nlp_model(callable_object):
    # Load the pre-trained BERT base model
    bert_layer = hub.KerasLayer(handle=callable_object, trainable=True)  
   
    # BERT layer three inputs: ids, masks and segments
    input_ids = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_ids")           
    input_masks = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_masks")       
    input_segments = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="segment_ids")
    
    inputs = [input_ids, input_masks, input_segments] # BERT inputs
    pooled_output, sequence_output = bert_layer(inputs) # BERT outputs
    
    # Add a hidden layer
    x = Dense(units=768, activation='relu')(pooled_output)
    x = Dropout(0.1)(x)
 
    # Add output layer
    outputs = Dense(2, activation="softmax")(x)

    # Construct a new model
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [42]:
model = nlp_model("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1")
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 500)]        0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 500)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 500)]        0           []                               
                                                                                                  
 keras_layer_1 (KerasLayer)     [(None, 768),        109482241   ['input_ids[0][0]',              
                                 (None, 500, 768)]                'input_masks[0][0]',      

In [43]:
tokenizer = create_tonkenizer(model.layers[3])

In [48]:
X_train = convert_sentences_to_features(x_train, tokenizer)
X_test = convert_sentences_to_features(x_test, tokenizer)

In [49]:
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

In [50]:
# Train the model
BATCH_SIZE = 16
EPOCHS = 1

# Use Adam optimizer to minimize the categorical_crossentropy loss
opt = Adam(learning_rate=2e-5)
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])

In [51]:
# Fit the data to the model
history = model.fit(X_train, Y_train,
                    validation_data=(X_test, Y_test),
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    verbose = 1)

251/251 [==============================] - 15064s 60s/step - loss: 0.3523 - accuracy: 0.8393 - val_loss: 0.2225 - val_accuracy: 0.9145


In [52]:
# Save the trained model
model_name = 'results/bert_model.h5'
model.save(model_name) 

In [53]:
# Load the pretrained nlp_model
new_model = load_model(model_name,custom_objects={'KerasLayer':hub.KerasLayer})

In [54]:
yhat_test = new_model.predict(X_test)

126/126 [==============================] - 2614s 21s/step


In [58]:
print(classification_report(np.argmax(Y_test,axis=1), np.argmax(yhat_test,axis=1)))

              precision    recall  f1-score   support

           0       0.93      0.90      0.91      2001
           1       0.90      0.93      0.92      2001

    accuracy                           0.91      4002
   macro avg       0.91      0.91      0.91      4002
weighted avg       0.91      0.91      0.91      4002



# Test mit neuen Daten

In [71]:
def sentiment(text):
    
    sentences = [text]
    
    features = convert_sentences_to_features( sentences, tokenizer)
    sentiment = model.predict(features)[0][1]
    
    comment = 'meh'
    if sentiment > 0.85:
        comment = 'very good'
    elif sentiment > 0.75:
        comment = 'good'
    elif sentiment > 0.50:
        comment = 'moderate'
    return sentiment,comment

In [72]:
test1 = "I simply don't like this film. I does not reach to any expectations."
print ( sentiment(test1))

1/1 [==============================] - 0s 373ms/step
(0.016892947, 'meh')


In [73]:
test1 = "I hate this film. It is the worst garbage on earth."
print ( sentiment(test1))

1/1 [==============================] - 0s 370ms/step
(0.015629997, 'meh')


In [74]:
test1 = "This is one of the best movies on earth."
print ( sentiment(test1))

1/1 [==============================] - 0s 371ms/step
(0.99177223, 'very good')


# Weiterführende Schritte


Stimmungsanalyse für Deutsch [https://machine-learning-blog.de/2019/06/03/stimmungsanalyse-sentiment-analysis-auf-deutsch-mit-python/](https://machine-learning-blog.de/2019/06/03/stimmungsanalyse-sentiment-analysis-auf-deutsch-mit-python/)

Anleitung für Zugriff auf twitter API [https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/](https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/)

<img src="info.png" align="left"/>
